### EVALUACIÓN CURSO DE EXTENSIÓN EN FINANZAS AVANZADAS (CEFA)- TÓPICOS DE PORTAFOLIOS 

Desarrollada por Esteban Camargo y Antonio Candia. 

A continuación va a encontrar los puntos de desarrollo en Python que se encuentran en el documento de word "Enunciado.docx". Por favor respondalos usando únicamente las celdas ya creadas en el notebook. Esto con el fin de facilitar su calificación. 

Por favor, no cree ni borre ninguna celda de la evaluación sea de Python o de Markdown. Si cree que le es útil para la solución de la evaluación, la recomendación sería desarrollar el código primero en un notebook de prueba y después si pasar a limpio su solución en el notebook que va a enviar por correo.

Cada punto se encuentra enunciado en una celda de markdown. A esta celda le siguen una celda de ejemplo y otra celda de solución. La celda de ejemplo inicia con el comentario "### Celda Ejemplo', esta celda proporciona instrucciones sobre la estructura de datos con la que contestar cada pregunta y como nombrarla. En la celda de solución que inicia con el comentario '### Celda Solución' usted debe incluir las líneas de código empleadas para resolver el punto, así como una variable nombrada igual que en la celda de ejemplo. Esta variable será utilizada para la calificación del examen. 

Por favor, proporcione su dirección de correo electrónico en la celda a continuación. Es importante incluirlo cuidadosamente, ya que está directamente vinculado al registro de su calificación:

In [57]:
correo = 'pabloperez12@gmail.com' # Cambiar por su correo electrónico

In [58]:
### Importación de Librerías

### En la siguiente sección se encuentra las librerías necesarias para desarrollar  
### el examen, se prefiere que use esas librerías frente a usar otras librerías disponibles en GitHub. 

import numpy as np
import pandas as pd
from skfolio.datasets import load_factors_dataset, load_sp500_dataset
from scipy.optimize import fmin
from scipy.cluster.hierarchy import linkage, dendrogram
from skfolio.preprocessing import prices_to_returns
from librerias.finance_operators import calculateLogReturns 
import scipy.stats 


from sklearn.model_selection import train_test_split
from skfolio import Population, RiskMeasure
from skfolio.datasets import load_sp500_dataset
from skfolio.optimization import InverseVolatility, RiskBudgeting

from skfolio.optimization import (
    EqualWeighted,
    HierarchicalRiskParity,
)
from skfolio.cluster import HierarchicalClustering, LinkageMethod

### Sección Práctica:

En el archivo “acciones.csv” se encuentra una serie de datos de acciones para el periodo comprendido entre 1999-12-31 y 2022-12-31.

En el archivo “yield 2019.csv” se encuentra las tasas de rendimientos del tesoro de los Estados Unidos para todo el año 2019. 

A continuación, use el archivo “yield 2019.csv” para desarrollar el punto número 1 del examen.

1.	Importe el archivo “yield 2019.csv”. Usando este archivo debe realizar la siguiente serie de pasos para interpolar la curva de rendimientos a la fecha del “2019-06-28” (10 puntos)

- Obtenga los parámetros $β_0$,$β_1$,$β_2$ y λ que minimizan el error cuadrático medio de la interpolación. Para ello use la función ‘getOptimalLoandings’ la cuál se trabajó en el notebook de la Lección 5. Modelos de Factores de Carga. 

- Use el modelo de Nelson y Siegel para obtener una interpolación de la curva de rendimientos del tesoro de los Estados Unidos en los nodos de 2 meses, 9 meses, 10 meses, 4 años y 6 años para la fecha del 2019-06-28.  

Su respuesta debe quedar en un dataframe titulado 'yield_df_punto1' con los índices y columnas como se muestran a continuación. La columna 'NS' debe ser resuelta con sus respectivos valores.

In [59]:
### Celda Ejemplo 
yield_df = pd.read_csv('Yield 2019.csv')
yield_df.index = yield_df['Date']
yield_df = yield_df.iloc[:,1:]
yield_series = yield_df.loc["2019-06-28",:]
def period_to_months(period):
    number, unit = period.split(' ')
    number = int(number)
    if unit == 'Yr':
        return number * 12
    return number


def standardizeYieldSeries(yield_series):
    yield_df = yield_series.to_frame()
    yield_df.columns = ['Yield']
    yield_df.loc['2 Mo',:] = np.nan
    yield_df.loc['9 Mo',:] = np.nan
    yield_df.loc['4 Yr',:] = np.nan
    yield_df.loc['6 Yr',:] = np.nan
    yield_df = yield_df.reset_index().rename(columns = {'index': 'TimePeriod'})
    yield_df['Yield'] = yield_df['Yield']/100
        
    
    yield_df['TimePeriodInMonths'] = yield_df['TimePeriod'].apply(period_to_months)
    yield_df = yield_df.sort_values('TimePeriodInMonths')
    yield_df['Maturity'] = yield_df['TimePeriodInMonths']/12
    # If you don't want to keep the 'TimePeriodInMonths' column, you can drop it
    yield_df = yield_df.reset_index(drop=True)
    yield_df
    return yield_df

yield_df_punto1 = standardizeYieldSeries(yield_series)
yield_df_punto1['NS'] = np.nan
yield_df_punto1
### Los valores en nan de la columna 'NS' deberían estar cargados con la respuesta

,TimePeriod,Yield,TimePeriodInMonths,Maturity,NS
0,2 Mo,NaN,2,0.166667,NaN
1,3 Mo,0.0212,3,0.250000,NaN
2,6 Mo,0.0209,6,0.500000,NaN
3,9 Mo,NaN,9,0.750000,NaN
4,1 Yr,0.0192,12,1.000000,NaN
5,2 Yr,0.0175,24,2.000000,NaN
6,3 Yr,0.0171,36,3.000000,NaN
7,4 Yr,NaN,48,4.000000,NaN
8,5 Yr,0.0176,60,5.000000,NaN
9,6 Yr,NaN,72,6.000000,NaN


In [60]:
### Celda Solución

A continuación, importe el archivo “acciones.csv” úselo para desarrollar los puntos 2 a 7 del examen. 

2.	Estime el retorno anual histórico de las acciones que se encuentran en el archivo "acciones.csv" para el periodo comprendido entre el 1999-12-31 al 2022-12-31 (10 puntos).  

Por favor reporte su solución en un pandas Series declarado como 'media_punto2' donde el índice sean las 20 acciones de la muestra.

In [61]:
### Celda Ejemplo
prices = pd.read_csv('acciones.csv')
prices.index = pd.to_datetime(prices['Date'])
prices = prices.iloc[:,1:]
media_punto2 = pd.Series(index = prices.columns)
media_punto2
### Los valores en nan deberían estar cargados con la respuesta

AAPL   NaN
AMD    NaN
BAC    NaN
BBY    NaN
CVX    NaN
GE     NaN
HD     NaN
JNJ    NaN
JPM    NaN
KO     NaN
LLY    NaN
MRK    NaN
MSFT   NaN
PEP    NaN
PFE    NaN
PG     NaN
RRC    NaN
UNH    NaN
WMT    NaN
XOM    NaN
dtype: float64

In [62]:
### Celda Solución

3.	Formule un intervalo de confianza para los retornos de las acciones especificando el error estándar. (10 puntos)

Por favor reporte su solución en un pandas DataFrame declarado como 'intervalo_punto3' donde el índice sean las 20 acciones de la muestra y las columnas sean el error estándar, el límite inferior y el límite superior. Use el nombre de las columnas que se muestra en el enunciado.

In [63]:
### Celda Ejemplo
intervalo_punto3 = pd.DataFrame(index = prices.columns, columns = ['SE', 'Limite_Inferior', 'Limite_Superior'])
intervalo_punto3
### Los valores en nan deberían estar cargados con la respuesta

,SE,Limite_Inferior,Limite_Superior
AAPL,NaN,NaN,NaN
AMD,NaN,NaN,NaN
BAC,NaN,NaN,NaN
BBY,NaN,NaN,NaN
CVX,NaN,NaN,NaN
GE,NaN,NaN,NaN
HD,NaN,NaN,NaN
JNJ,NaN,NaN,NaN
JPM,NaN,NaN,NaN
KO,NaN,NaN,NaN


In [64]:
### Celda Solución

Responda las preguntas 4, 5, 6 y el bono usando la librería ‘Skfolio’. No es válido usar otras librerías de optimización de portafolios. 

4.	Seccione los datos entre entrenamiento (‘train’) y prueba (‘test’) usando el 60% de los datos disponibles para entrenamiento. (10 puntos)

Por favor cree dos dataframes X_train y X_test

In [65]:
### Celda Ejemplo
X_train_punto4 = pd.DataFrame(columns = prices.columns, index = prices.index)
X_train_punto4
X_test_punto4 = pd.DataFrame(columns = prices.columns, index = prices.index)
X_test_punto4
### Los valores en nan deberían estar cargados con la respuesta

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
1999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
### Solución

5.	Obtenga el portafolio de Risk Parity, Hierarchical Risk Parity e Inverse Volatility Porfolio. Utilice como métrica de riesgo la varianza y el resto de los parámetros especificados por default (10 puntos). 

Por favor cree un dataframes donde los índices sean el peso que se le da dentro de la muestra a cada acción y las columnas sean los métodos de optimización.

In [67]:
### Celda Ejemplo
weights_df_punto5 = pd.DataFrame(index = prices.columns, columns = ['Risk Parity - Variance', 'Inverse Vol', 'HierarchicalRiskParity'])
weights_df_punto5

,Risk Parity - Variance,Inverse Vol,HierarchicalRiskParity
AAPL,NaN,NaN,NaN
AMD,NaN,NaN,NaN
BAC,NaN,NaN,NaN
BBY,NaN,NaN,NaN
CVX,NaN,NaN,NaN
GE,NaN,NaN,NaN
HD,NaN,NaN,NaN
JNJ,NaN,NaN,NaN
JPM,NaN,NaN,NaN
KO,NaN,NaN,NaN


In [68]:
### Solución

6.	Obtenga los Analíticos financieros de los anteriores tres portafolios tanto dentro de la muestra (‘In-Sample’) como fuera de la misma (‘Out-of-Sample’). Si la función de utilidad es la siguiente:

$ U = Annualized Sharpe Ratio_{out-of-sample}+0.5 CalmarRatio_{out-of-sample} $

¿Cuál portafolio escogería el inversionista entre los tres métodos?

Por favor cree un string llamado respuesta_punto6 usando la lista que está en la celda.

In [69]:
### Celda Ejemplo
opciones_respuesta = ['Risk Parity - Variance', 'Inverse Vol', 'HierarchicalRiskParity']
respuesta_punto6 = opciones_respuesta[1]
respuesta_punto6

'Inverse Vol'

In [70]:
### Solución

Bono: Obtenga un portafolio que maximice el retorno esperado del inversionista, suponiendo que este pondera los retornos con la siguiente función de utilidad: 

Un ponderado de 50% el retorno dentro de la muestra “In-Sample” y un 50% de retorno fuera de la muestra “Out-of-Sample”. A continuación, la función de utilidad definida:

$U=0.5r_{is}+0.5r_{oos}$

No se permite resolver el problema por fuerza bruta. Puede usar otros métodos de optimización como media varianza o Black-Litterman. Se ponderará el bono según su desempeño relativo al resto de los estudiantes. 

Por favor cree un objeto del tipo population nombrado 'population_bono_is' que contenga solamente las predicciones dentro de la muestra del portafolio que seleccionó para obtener la mayor utilidad posible. 

Adicionalmente, cree un objeto del tipo 'population_bono_oos' que contenga solamente las predicciones fuera de la muestra del portafolio que seleccionó para obtener la mayor utilidad posible. 

Por favor no incluya las comparaciones que pudo haber hecho con otros modelos de optimización en los objetos population de respuesta.
 
Por último, evalué la función de utilidad con el método calcularUtilidadBono() y guarde el resultado en la variable "utilidad_bono", adicionalmente proporcione el nombre del metódo seleccionado por usted en la variable "nombre_metodo_bono"


In [71]:
def convert_to_float(val):
    try:
        if '%' in val:
            return float(val.strip('%')) / 100
        return float(val)
    except ValueError:
        return val
    
def calcularUtilidadBono(population_bono_oos, population_bono_is):
    """
    Parámetros:

    population_bono_oos (DataFrame): Objeto de la clase Population "out of sample".
    population_bono_is (DataFrame): Objeto de la clase Population "in sample".

    Retorna:
    float: La utilidad calculada del portafolio.

    """
    population_bono_oos_df = population_bono_oos.summary()
    population_bono_oos_df = population_bono_oos_df.map(convert_to_float)

    population_bono_is_df = population_bono_is.summary()
    population_bono_is_df = population_bono_is_df.map(convert_to_float)
  
    utilidad = 0.5 * population_bono_oos_df.loc['Annualized Mean', :].values[0] + 0.5 * population_bono_is_df.loc['Annualized Mean', :].values[0]
    utilidad = utilidad.round(3)
    
    return utilidad

In [72]:
### Celda Ejemplo

utilidad_bono = 0 
nombre_metodo_bono = 'Risk Parity - Variance'

# En este caso, no se pueden crear las instancias de los objetos "population_bono_oos" y "population_bono_is" para fines de ejemplificación. 
# No obstante, usted deberá generar estos objetos con los nombres adecuados durante la implementación de la solución.

In [73]:
### Solución